In [98]:
import sys
import os
os.environ['USE_PYGEOS'] = '0'
import gc
import collections

import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams, cycler
from matplotlib import animation, rc
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import geopandas as gpd

import numpy as np
import xarray as xr
# from xeofs.xarray import EOF
import rioxarray

import scipy
from scipy import signal
import cftime
from shapely.geometry import mapping
from xarrayutils.utils import linear_trend, xr_linregress
import pandas as pd

In [99]:
# File path directories

inDirName = '/Users/smurugan9/research/aislens/aislens_emulation/'
DIR_external = 'data/external/'

# DATASET FILEPATHS
# Basal melt observations from Paolo 2023
DIR_basalMeltObs = 'data/external/Paolo2023/'
# Ocean model output - E3SM (SORRMv2.1.ISMF), data received from Darin Comeau / Matt Hoffman at LANL
DIR_SORRMv21 = 'data/external/SORRMv2.1.ISMF/regridded_output/'

# INTERIM GENERATED FILEPATHS
DIR_basalMeltObs_Interim = 'data/interim/Paolo2023/iceShelves_dedraft/'


# DATA FILENAMES
FILE_basalMeltObs = 'ANT_G1920V01_IceShelfMelt.nc'
FILE_MeltDraftObs = 'ANT_G1920V01_IceShelfMeltDraft.nc'
FILE_SORRMv21 = 'Regridded_SORRMv2.1.ISMF.FULL.nc'


FILE_iceShelvesShape = 'iceShelves.geojson'

In [100]:
MELTDRAFT_OBS = xr.open_dataset(inDirName+DIR_basalMeltObs+FILE_MeltDraftObs)
SORRMv21 = xr.open_dataset(inDirName+DIR_SORRMv21+FILE_SORRMv21)

ICESHELVES_MASK = gpd.read_file(inDirName+DIR_external+FILE_iceShelvesShape)
icems = ICESHELVES_MASK.to_crs({'init': 'epsg:3031'});
crs = ccrs.SouthPolarStereo();

/Users/smurugan9/opt/anaconda3/envs/aislens/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [13]:
# SORRMv21_ts = np.empty(SORRMv21.timeMonthly_avg_landIceFreshwaterFlux.shape[0])
# for i in range(SORRMv21_ts.shape[0]):
#     SORRMv21_ts[i] = SORRMv21.timeMonthly_avg_landIceFreshwaterFlux[i].mean(['x','y'])

In [30]:
sorrm_ts = SORRMv21.timeMonthly_avg_landIceFreshwaterFlux.mean(['x','y'])

In [104]:
MELTDRAFT_OBS.rio.write_crs("epsg:3031",inplace=True);
h = MELTDRAFT_OBS.draft
h_mean = h.mean('time')

In [ ]:
for i in range(6,32):
    print('extracting data for catchment {}'.format(icems.name.values[i]))
    mlt = MELTDRAFT_OBS.melt.rio.clip(icems.loc[[i],'geometry'].apply(mapping),icems.crs,drop=False)
    mlt_mean = mlt.mean('time')
    #h = ds.timeMonthly_avg_ssh
    #h_mean = h.mean('time')
    # Dedraft: Linear Regression with SSH over chosen basin
    print('calculating linear regression for catchment {}'.format(icems.name.values[i]))
    mlt_rgrs = xr_linregress(h, mlt_mean, dim='time') # h = independent variable
    mlt_prd = mlt_rgrs.slope*h_mean + mlt_rgrs.intercept
    #flx_ddrft = flx - flx_prd
    mlt_prd.to_netcdf(inDirName+DIR_basalMeltObs_Interim+'{}_rgrs.nc'.format(icems.name.values[i]))
    print('{} file saved'.format(icems.name.values[i]))
    del mlt, mlt_mean, mlt_rgrs, mlt_prd
    print('deleted interim variables')
    gc.collect()

extracting data for catchment IMBIE1
calculating linear regression for catchment IMBIE1
IMBIE1 file saved
deleted interim variables
extracting data for catchment IMBIE2
calculating linear regression for catchment IMBIE2
IMBIE2 file saved
deleted interim variables
extracting data for catchment IMBIE3
calculating linear regression for catchment IMBIE3
IMBIE3 file saved
deleted interim variables
extracting data for catchment IMBIE4
calculating linear regression for catchment IMBIE4
IMBIE4 file saved
deleted interim variables
extracting data for catchment IMBIE5
calculating linear regression for catchment IMBIE5
IMBIE5 file saved
deleted interim variables
extracting data for catchment IMBIE6
calculating linear regression for catchment IMBIE6
IMBIE6 file saved
deleted interim variables
extracting data for catchment IMBIE7
calculating linear regression for catchment IMBIE7
IMBIE7 file saved
deleted interim variables
extracting data for catchment IMBIE8
calculating linear regression for catch

In [47]:
# IMBIE ice shelves = 6 to 32

'IMBIE1'

In [ ]:
# Save linear regression interim files for MELT_OBS and SORRMv21

In [ ]:
# Plot dedrafted melt for MELT_OBS and SORRMv21 across ice shelves.
# To save time, can use IMBIE definitions of ice shelves at first - 
# these are larger catchments with mostly similar behavior as the 
# refined catchment definitions of individual ice shelves

In [ ]:
# Compare PAOLO23 to SORRMv21
# Spatiotemporal means in a scatterplot

/Users/smurugan9/opt/anaconda3/envs/aislens/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [49]:
# Refer Paolo 2023, or captoolkit
# Lines 260-261 in 
# https://github.com/nasa-jpl/captoolkit/blob/be9aa159f2eec912e55d7b031f16ac026c8bd045/captoolkit/cubethick.py#L260C66-L260C66

# D = (1 - (rho_ocean - rho_ice) / rho_ocean) H

# H_draft = H_freeb * ((rho_ocean / (rho_ocean - rho_ice)) - 1)
# H = H_freeb * rho_ocean / (rho_ocean - rho_ice)

H = melt.thickness
rho_ocean = 1028.0
rho_ice = 917.0
H_draft = (H*(rho_ocean-rho_ice)/rho_ocean)*((rho_ocean / (rho_ocean - rho_ice)) - 1)
melt['draft'] = H_draft


# xarray raises an issue when saving the modified dataset with just the melt and draft variables, 
# delete grid_mapping attribute to proceed with writing the file
# del ds.melt.attrs['grid_mapping']

ds1 = melt.melt
ds2 = melt.draft
ds = xr.merge([ds1,ds2])
ds.to_netcdf(inDirName+DIR_basalMeltObs+'ANT_G1920V01_IceShelfMeltDraft.nc')

In [76]:
mlt = melt.melt.rio.clip(icems.loc[[i],'geometry'].apply(mapping),icems.crs,drop=False)
mlt_mean = mlt.mean('time')
#h = ds.timeMonthly_avg_ssh
#h_mean = h.mean('time')
# Dedraft: Linear Regression with SSH over chosen basin
print('calculating linear regression for catchment {}'.format(icems.name.values[i]))
mlt_rgrs = xr_linregress(h, mlt_mean, dim='time') # h = independent variable
mlt_prd = mlt_rgrs.slope*h_mean + mlt_rgrs.intercept
#flx_ddrft = flx - flx_prd

calculating linear regression for catchment IMBIE1


extracting data for catchment IMBIE1
calculating linear regression for catchment IMBIE1
IMBIE1 file saved
deleted interim variables
extracting data for catchment IMBIE2
calculating linear regression for catchment IMBIE2
IMBIE2 file saved


NameError: name 'ds' is not defined

In [85]:
# Scatterplots of slope of ice shelves in Paolo vs. SORRM
# Scatterplots of ice shelf spatiotemporal means in dedrafted version

'data/interim/Paolo2023/iceShelves_dedraft/'